In [7]:
import pandas as pd
import cloudscraper
from urllib.request import urlopen
from bs4 import BeautifulSoup
from IPython.display import clear_output


In [8]:
%run ./init.ipynb

Length of attributes added  100000
Number of valid rows with attributes  96406


In [9]:
listing = {}

In [10]:
%%time
startRange = 96405
stopRange = 97000

refStartx = startRange
for x in range(startRange, stopRange):
    scraper = cloudscraper.create_scraper(
        delay=1,
        browser={
            'browser': 'firefox',
            'platform': 'windows',
            'mobile': False
        })
    html = scraper.get("https://niftys.com/nft/0x39ceaa47306381b6d79ad46af0f36bc5332386f2/" + str(x)).text

    soup = BeautifulSoup(html, 'lxml')

    rows = soup.find_all('div', 'label')

    rowAttributes = {}
    for row in rows:
        cells = row
        str_row = str(row)
        value = str_row.split('!-- -->')
        cleanTextAttribute = BeautifulSoup(value[0], "lxml").get_text()
        if cleanTextAttribute == 'Available on PALM':
            continue
        if (len(value) > 1):
            cleanTextValue = BeautifulSoup(value[1], "lxml").get_text().strip()
        else:
            cleanTextValue = "None"
        
        rowAttributes[cleanTextAttribute] = cleanTextValue
    
    output_list = [(rowAttributes[key]) for key in (manualColumns) if key in rowAttributes]

    
    listing[str(x)] = output_list
    clear_output(wait=True)
    print(x)
    if(x % 5000 == 0):
        if refStartx == 1:
            refStartx = 0
        data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColums)  #pd.DataFrame(listing, orient='index')
        data_dictionary.to_csv('Attributes_' + str(refStartx + 1) + '_' + str(x) + '.csv')
        refStartx = x
        listing.clear()

96999
CPU times: user 54.1 s, sys: 3.25 s, total: 57.4 s
Wall time: 20min 2s


In [13]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColumns)  #pd.DataFrame(listing, orient='index')
data_dictionary.to_csv('Attributes_' + str(refStartx + 1) + '_' + str(x) + '.csv')

In [11]:
data_dictionary =  pd.DataFrame.from_dict(listing, orient='index', columns=manualColums)  #pd.DataFrame(listing, orient='index')

NameError: name 'manualColums' is not defined

In [ ]:
data_dictionary

In [ ]:
for count in range(0, len(manualColums)):
    print(count)
    print(data_dictionary[manualColums[count]].value_counts())
    data_dictionary[manualColums[count]].value_counts(normalize=True)
    print('---------------------------------')

In [ ]:
data_dictionary.to_csv('Attributes_' + str(startRange) + '_' + str(stopRange-1) + '.csv')

In [ ]:
print('done')